In [11]:
import pandas as pd
import re
import json
from bs4 import BeautifulSoup
import rdata
import numpy as np

In [12]:
nacc_test = pd.read_csv(r'C:\Users\jaiga\Downloads\UDS3Mod_for_Jai_Crosswalk_Levels_only.csv')
nacc_test.head(3)

,record_id,redcap_event_name,redcap_repeat_instrument,redcap_repeat_instance,redcap_data_access_group,redcap_survey_identifier,adc_sub_id,visitmo,visitday,visityr,...,pbmc_count,pbmc_dt,plasma_count,plasma_dt,serum_count,serum_dt,urine_count,urine_dt,biospe_dde_notes,biospecimen_inventory_complete
0,104,visit_info_arm_1,NaN,1,crosswalk_data,NaN,ADC1105,1,14,2025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,105,visit_info_arm_1,NaN,1,crosswalk_data,NaN,ADC1102,1,13,2025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,106,visit_info_arm_1,NaN,1,crosswalk_data,NaN,ADC1129,2,5,2025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
nacc_test.shape

(3, 2900)

In [14]:
nacc_test['hisporx'].value_counts()

hisporx
Guatemalan    1
Name: count, dtype: int64

In [15]:
# Load JSON mapping file
with open(r'C:\Users\jaiga\Downloads\crosswalks\A1_UDS4_programmatic_crosswalk_mappings.json', 'r', encoding='utf-8') as file:
    mapping_data = json.load(file)

In [16]:
mapping_data.keys()

dict_keys(['Direct_Mappings', 'Conditional_Consistency', 'Structured_Transformations', 'High_Complexity'])

In [44]:
uds3_df = nacc_test
uds3_df.columns = uds3_df.columns.str.lower()
uds4_df = pd.DataFrame()

data_dict = {'ptid__a1': 'adc_sub_id', 'visitnum__a1': 'a1_visit_day','initials__a1':'a1_ex_ini'}

In [45]:
uds3_df.shape

(3, 2900)

In [46]:
uds3_df['race'].value_counts()

race
2    2
1    1
Name: count, dtype: int64

In [54]:
def process_mappings(mapping_type):
    for mapping in mapping_data.get(mapping_type, []):
        uds3_var = mapping["UDS3_variable"].lower() 
        uds4_var = mapping["UDS4_variable"].lower()
        
        uds3_var = data_dict.get(uds3_var, uds3_var)
        
        if mapping_type == 'Direct_Mappings':
            response_levels = [m for m in mapping["crosswalk_remappings"] if m["mapping_type"] == "Response LEVELS"]
            conformity = [m for m in mapping["crosswalk_remappings"] if m["mapping_type"] == "Conformity"]
            
        else:
            response_levels = [m for m in mapping["crosswalk_mappings"] if m["mapping_type"] == "Response LEVELS"]
            conformity = [m for m in mapping["crosswalk_mappings"] if m["mapping_type"] == "Conformity"]
        
       # Case 1: No Response LEVELS, copy values directly
        if not response_levels:
            if uds3_var in uds3_df.columns:
                    uds4_df[uds4_var] = uds3_df[uds3_var]
        
        # Case 2: Conformity check
        elif conformity:
            uds3_conformity_values = set(item["UDS3_value"] for mapping in conformity for item in mapping["mappings"])
            uds4_conformity_values = set(item["UDS4_value"] for mapping in conformity for item in mapping["mappings"])
            
            if uds3_conformity_values == uds4_conformity_values:  # Ensure conformity matches
                if uds3_var in uds3_df.columns:
                    uds4_df[uds4_var] = uds3_df[uds3_var]
            else:
                response_map = {str(item["UDS3_value"]): str(item["UDS4_value"]) for mapping in response_levels for item in mapping["mappings"]}
                
                if uds3_var in uds3_df.columns:
                    # Applying exact single-value mappings first (e.g., 2 -> 1)
                    for uds3_value, uds4_value in response_map.items():
                        if "|" not in uds3_value and "grep(" not in uds3_value:
                            uds4_df.loc[uds3_df[uds3_var].astype(str) == uds3_value, uds4_var] = uds4_value
                            
                        else:
                            if not (uds3_var.endswith("sec") or uds3_var.endswith("ter")): #ignoring rest of racesec since it wont add any value
                                # Handling cases with logical OR conditions (e.g., "1 | 3 | 4 | 5 | 50 | 99" -> None)
                                values = uds3_value.split(" | ")
                                uds4_df.loc[uds3_df[uds3_var].astype(str).isin(values), uds4_var] = uds4_value
                    
                    # Handling grep-based text searches (e.g., grep("guatemalan", HISPORX))
                    for uds3_value, uds4_value in response_map.items():
                        if "grep(" in uds3_value:
                            search_term = re.search(r'grep\("(.*?)",', uds3_value).group(1).lower()
                            uds4_df.loc[uds3_df[uds3_var].astype(str).str.contains(search_term, case=False, na=False), uds4_var] = uds4_value
        
        # Case 3: Handling secondary and tertiary race variables (race, racesec, raceter)
        if uds3_var.endswith("sec") or uds3_var.endswith("ter"):
            if uds3_var in uds3_df.columns:
                missing_mask = uds4_df[uds4_var] == "NA"  # Identify rows where primary race is NA
                
                # Only update NA values in UDS4 based on racesec or raceter mappings
                for uds3_value, uds4_value in response_map.items():
                    if "|" not in uds3_value and "grep(" not in uds3_value:
                        uds4_df.loc[missing_mask & (uds3_df[uds3_var].astype(str) == uds3_value), uds4_var] = uds4_value
                    elif "grep(" in uds3_value:
                        search_term = re.search(r'grep\("(.*?)",', uds3_value).group(1).lower()
                        uds4_df.loc[missing_mask & uds3_df[uds3_var].astype(str).str.contains(search_term, case=False, na=False), uds4_var] = uds4_value
                    else:
                        values = uds3_value.split(" | ")
                        uds4_df.loc[missing_mask & uds3_df[uds3_var].astype(str).isin(values), uds4_var] = uds4_value

# Process different mapping types dynamically
mapping_types = ["Direct_Mappings", "Conditional_Consistency"]  # Add more types as needed
for mapping_type in mapping_types:
    process_mappings(mapping_type)

In [ ]:
# UTF-8-SIG encoding to avoid character corruption
uds4_df.to_csv("uds4_data11.csv", index=False, na_rep="NA", encoding='utf-8-sig')

In [55]:
uds4_df.head(3)

,birthmo,educ,handed,ethhisothx,livsitua,maristat,ptid__a1,refersc,birthsex,sourcenw,...,initialsa1,predomlan,raceaian,raceasian,raceblack,racenhpi,raceunkn,racewhite,racemena,residenc
0,2,12,2,Guatemalan,6,2,ADC1105,2,2,1,...,zz,1,None,None,None,None,None,1,NaN,3
1,3,12,2,NaN,1,2,ADC1102,3,1,1,...,cc,1,None,None,1,1,None,None,NaN,1
2,5,16,2,NaN,2,1,ADC1129,4,2,1,...,CVB,1,None,None,1,1,None,None,NaN,1
